In [1]:
from scipy.io import loadmat
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from ndsys.optimizers import EntropicDualAveraging
from ndsys.features.volterra import VolterraFeatures, prepare_data

Load data recorded from the cascaded tanks system (the benchmark was downloaded from https://www.nonlinearbenchmark.org/benchmarks/cascaded-tanks; see the link for more details)

In [2]:
data = loadmat('dataBenchmark.mat')

x_est = data['uEst']
y_est = data['yEst'].ravel()
x_val = data['uVal']
y_val = data['yVal'].ravel()

Set parameters of the model and the algorithm

In [3]:
kernels = (100, 100, 20)
R = 25
learning_rate = 'adaptive'

Since we handle data recorded from a dynamical system, an additional preprocessing step is needed

In [4]:
x_est, y_est = prepare_data(x_est, y_est, kernels)
x_val, y_val = prepare_data(x_val, y_val, kernels, x0='zeros')

Create a pipeline

In [5]:
est = make_pipeline(MaxAbsScaler(),
                    VolterraFeatures(kernels),
                    EntropicDualAveraging(l1_constraint=R, learning_rate=learning_rate))

Fit and predict!

In [ ]:
est.fit(x_est, y_est)
y_pred = est.predict(x_val)

Finally, we can plot the data

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
plt.rcdefaults()
plt.rcParams['text.usetex'] = True
plt.rc('font', family='serif')
plt.rc('font', size=10)
plt.rc('axes', labelsize='small')
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')
plt.rc('legend', fontsize='small')

plt.figure()

plt.plot(y_pred)
plt.plot(y_val, '--')
plt.xlabel('t')
plt.ylabel('output')
plt.legend(['Model', 'True system'])
plt.grid()
plt.show()